<a href="https://colab.research.google.com/github/Kritdanai044/Data_movies/blob/main/Movies_Cleansing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt-get update                                                                          # อัพเดท Package ทั้งหมดใน VM ตัวนี้
!apt-get install openjdk-8-jdk-headless -qq > /dev/null                                  # ติดตั้ง Java Development Kit (จำเป็นสำหรับการติดตั้ง Spark)
!wget -q https://archive.apache.org/dist/spark/spark-3.1.2/spark-3.1.2-bin-hadoop2.7.tgz # ติดตั้ง Spark 3.1.2
!tar xzvf spark-3.1.2-bin-hadoop2.7.tgz                                                  # Unzip ไฟล์ Spark 3.1.2
!pip install -q findspark==1.3.0                                                         # ติดตั้ง Package Python สำหรับเชื่อมต่อกับ Spark 
!pip install gdown

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:5 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:13 http://security.ubuntu.com/ubuntu bionic-security/main amd64 Packages [2,990 kB]
Hit:14 http://ppa.l

In [2]:
# Set enviroment variable ให้ Python รู้จัก Spark
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop2.7"

In [3]:
# ติดตั้ง PySpark ลงใน Python
!pip install pyspark==3.1.2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 212.4 MB 69 kB/s 
     |████████████████████████████████| 198 kB 22.8 MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.2-py2.py3-none-any.whl size=212880769 sha256=773c43aa1de00e21a49c9a3bda4d509648e6f16ee1c9a328d99842526224ad76
  Stored in directory: /root/.cache/pip/wheels/a5/0a/c1/9561f6fecb759579a7d863dcd846daaa95f598744e71b02c77
Successfully built pyspark


In [4]:
# สร้าง Spark Session เพิ้อใช้งาน Spark
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [5]:
from pyspark.sql.functions import when
from pyspark.sql.functions import mean
from pyspark.sql.functions import avg

In [6]:
from pyspark.sql import functions as f
import gdown

In [14]:
!gdown  https://drive.google.com/uc?id=1lfBoXhtJcFfUp2lj0RiHCKQn1n1kQREy

Downloading...
From: https://drive.google.com/uc?id=1lfBoXhtJcFfUp2lj0RiHCKQn1n1kQREy
To: /content/output.csv
100% 3.41M/3.41M [00:00<00:00, 157MB/s]


In [15]:
dt = spark.read.csv('/content/output.csv', header = True, inferSchema = True, )

In [16]:
dt

DataFrame[movieId: int, title: string, genres: string, rating: double]

In [17]:
dt.dtypes

[('movieId', 'int'),
 ('title', 'string'),
 ('genres', 'string'),
 ('rating', 'double')]

In [11]:
dt.show(20)

+--------------------+
|     <!DOCTYPE html>|
+--------------------+
|    <html lang="en">|
|              <head>|
|    <meta charset...|
|    <meta content...|
|    <meta content...|
|    <meta http-eq...|
|    <meta name="r...|
|    <link rel="ap...|
|    <link rel="ic...|
|    <link rel="ic...|
|    <link rel="sh...|
|    <link rel="st...|
|    <link href="h...|
|    <link href="/...|
|    <meta name="t...|
|    <meta name="t...|
|    <meta name="t...|
|    <meta name="t...|
|    <meta name="t...|
|    <meta propert...|
+--------------------+
only showing top 20 rows



In [ ]:
# นับจำนวนแถวและ column
print((dt.count(), len(dt.columns)))

(58098, 4)


In [ ]:
58098-53889

4209

In [ ]:
# สรุปข้อมูลสถิติ
dt.describe().show()

+-------+------------------+--------------------+------------------+------------------+
|summary|           movieId|               title|            genres|            rating|
+-------+------------------+--------------------+------------------+------------------+
|  count|             58098|               58098|             58098|             53889|
|   mean|111919.51619677097|                null|              null|3.0685927253973246|
| stddev|59862.660955928804|                null|              null|0.7362424202405303|
|    min|                 1|"""Great Performa...|(no genres listed)|               0.5|
|    max|            193886|     줄탁동시 (2012)|           Western|               5.0|
+-------+------------------+--------------------+------------------+------------------+



In [ ]:
# อีกคำสั่งในการดูข้อมูลแต่ละคอลัมน์ (Schema)
dt.printSchema()

root
 |-- movieId: integer (nullable = true)
 |-- title: string (nullable = true)
 |-- genres: string (nullable = true)
 |-- rating: double (nullable = true)



In [ ]:
dt.filter(dt.rating.isNull()).show()

+-------+--------------------+--------------------+------+
|movieId|               title|              genres|rating|
+-------+--------------------+--------------------+------+
|  25817|Break of Hearts (...|       Drama|Romance|  null|
|  26361|Baby Blue Marine ...|               Drama|  null|
|  27153|Can't Be Heaven (...|Children|Comedy|D...|  null|
|  27433|        Bark! (2002)|        Comedy|Drama|  null|
|  31945|Always a Bridesma...|         Documentary|  null|
|  52696|Thousand and One ...|           Adventure|  null|
|  58209|Alex in Wonder (S...|        Comedy|Drama|  null|
|  60234|   Shock, The (1923)|         Crime|Drama|  null|
|  69565|Bling: A Planet R...|         Documentary|  null|
|  69834|       Agency (1980)|               Drama|  null|
|  69864|   Blue Blood (2006)|         Documentary|  null|
|  70312|Family Secret, A ...|Comedy|Drama|Musical|  null|
|  71355|February (Khumpha...|       Drama|Romance|  null|
|  72901|Art of Vision, Th...|               Drama|  nul

In [ ]:
dt.select(mean("rating")).show()

+------------------+
|       avg(rating)|
+------------------+
|3.0685927253973246|
+------------------+



In [ ]:
dt_clean_null = dt.withColumn("c_rating", when(dt['rating'].isNull(), 3.06).otherwise(dt['rating']))
dt_clean =dt_clean_null.drop("rating").withColumnRenamed('c_rating', 'rating')

In [ ]:
dt_clean.show(20)

+-------+--------------------+--------------------+------------------+
|movieId|               title|              genres|            rating|
+-------+--------------------+--------------------+------------------+
|      1|    Toy Story (1995)|Adventure|Animati...|3.8866494325899312|
|      2|      Jumanji (1995)|Adventure|Childre...| 3.246582912721512|
|      3|Grumpier Old Men ...|      Comedy|Romance| 3.173981392364453|
|      4|Waiting to Exhale...|Comedy|Drama|Romance| 2.874539979926397|
|      5|Father of the Bri...|              Comedy|3.0772909396406876|
|      6|         Heat (1995)|Action|Crime|Thri...|3.8442108566049575|
|      7|      Sabrina (1995)|      Comedy|Romance| 3.371348277890334|
|      8| Tom and Huck (1995)|  Adventure|Children| 3.122482131254061|
|      9| Sudden Death (1995)|              Action| 3.007529781973477|
|     10|    GoldenEye (1995)|Action|Adventure|...| 3.431632714743396|
|     11|American Presiden...|Comedy|Drama|Romance| 3.660277594183741|
|     

In [ ]:
dt_clean.describe().show()

+-------+------------------+--------------------+------------------+------------------+
|summary|           movieId|               title|            genres|            rating|
+-------+------------------+--------------------+------------------+------------------+
|  count|             58098|               58098|             58098|             58098|
|   mean|111919.51619677097|                null|              null|3.0679702120370176|
| stddev|59862.660955928804|                null|              null|0.7090749757950141|
|    min|                 1|"""Great Performa...|(no genres listed)|               0.5|
|    max|            193886|     줄탁동시 (2012)|           Western|               5.0|
+-------+------------------+--------------------+------------------+------------------+



In [ ]:
dt_clean.filter(dt_clean.rating.isNull()).show()

+-------+-----+------+------+
|movieId|title|genres|rating|
+-------+-----+------+------+
+-------+-----+------+------+



In [ ]:
from pyspark.sql.functions import col
from pyspark.sql.types import StringType,BooleanType,DateType

In [ ]:
dt_clean = dt_clean.withColumn("movieId",col("movieId").cast(StringType()))

In [ ]:
dt_clean.printSchema()

root
 |-- movieId: string (nullable = true)
 |-- title: string (nullable = true)
 |-- genres: string (nullable = true)
 |-- rating: double (nullable = true)



In [ ]:
dt_clean.show()

+-------+--------------------+--------------------+------------------+
|movieId|               title|              genres|            rating|
+-------+--------------------+--------------------+------------------+
|      1|    Toy Story (1995)|Adventure|Animati...|3.8866494325899312|
|      2|      Jumanji (1995)|Adventure|Childre...| 3.246582912721512|
|      3|Grumpier Old Men ...|      Comedy|Romance| 3.173981392364453|
|      4|Waiting to Exhale...|Comedy|Drama|Romance| 2.874539979926397|
|      5|Father of the Bri...|              Comedy|3.0772909396406876|
|      6|         Heat (1995)|Action|Crime|Thri...|3.8442108566049575|
|      7|      Sabrina (1995)|      Comedy|Romance| 3.371348277890334|
|      8| Tom and Huck (1995)|  Adventure|Children| 3.122482131254061|
|      9| Sudden Death (1995)|              Action| 3.007529781973477|
|     10|    GoldenEye (1995)|Action|Adventure|...| 3.431632714743396|
|     11|American Presiden...|Comedy|Drama|Romance| 3.660277594183741|
|     

In [ ]:
dt_clean = dt_clean.withColumn("new_rate", f.round(dt_clean["rating"], 2)).withColumnRenamed('new_rate', 'avg_rating')
dt_clean =dt_clean.drop("rating")

In [ ]:
dt_clean.show()

+-------+--------------------+--------------------+----------+
|movieId|               title|              genres|avg_rating|
+-------+--------------------+--------------------+----------+
|      1|    Toy Story (1995)|Adventure|Animati...|      3.89|
|      2|      Jumanji (1995)|Adventure|Childre...|      3.25|
|      3|Grumpier Old Men ...|      Comedy|Romance|      3.17|
|      4|Waiting to Exhale...|Comedy|Drama|Romance|      2.87|
|      5|Father of the Bri...|              Comedy|      3.08|
|      6|         Heat (1995)|Action|Crime|Thri...|      3.84|
|      7|      Sabrina (1995)|      Comedy|Romance|      3.37|
|      8| Tom and Huck (1995)|  Adventure|Children|      3.12|
|      9| Sudden Death (1995)|              Action|      3.01|
|     10|    GoldenEye (1995)|Action|Adventure|...|      3.43|
|     11|American Presiden...|Comedy|Drama|Romance|      3.66|
|     12|Dracula: Dead and...|       Comedy|Horror|      2.67|
|     13|        Balto (1995)|Adventure|Animati...|    

In [ ]:
dt_clean.write.csv('Cleaned_data.csv', header = True)

In [ ]:
dt_clean.coalesce(1).write.csv('Cleaned_Data_Single.csv', header = True)

In [ ]:
dt_clean.show()

+-------+--------------------+--------------------+----------+
|movieId|               title|              genres|avg_rating|
+-------+--------------------+--------------------+----------+
|      1|    Toy Story (1995)|Adventure|Animati...|      3.89|
|      2|      Jumanji (1995)|Adventure|Childre...|      3.25|
|      3|Grumpier Old Men ...|      Comedy|Romance|      3.17|
|      4|Waiting to Exhale...|Comedy|Drama|Romance|      2.87|
|      5|Father of the Bri...|              Comedy|      3.08|
|      6|         Heat (1995)|Action|Crime|Thri...|      3.84|
|      7|      Sabrina (1995)|      Comedy|Romance|      3.37|
|      8| Tom and Huck (1995)|  Adventure|Children|      3.12|
|      9| Sudden Death (1995)|              Action|      3.01|
|     10|    GoldenEye (1995)|Action|Adventure|...|      3.43|
|     11|American Presiden...|Comedy|Drama|Romance|      3.66|
|     12|Dracula: Dead and...|       Comedy|Horror|      2.67|
|     13|        Balto (1995)|Adventure|Animati...|    

**เนื่องจากนำเข้า Big query ไม่ได้จึงหาวิธีแก้ไข**

In [ ]:
dt_clean.where(dt_clean["title"].like('%"%')).show()

+-------+--------------------+--------------------+----------+
|movieId|               title|              genres|avg_rating|
+-------+--------------------+--------------------+----------+
|   7789|"11'09""01 - Sept...|               Drama|      3.37|
|  51372|"""Great Performa...|             Musical|      2.82|
| 112809|"Diebuster ""Top ...|Action|Animation|...|      3.38|
| 138808|"The Death of ""S...|         Documentary|      3.25|
| 174029|"Divers at Work o...|  (no genres listed)|       3.0|
| 182853|"How Viktor ""The...|               Drama|      3.72|
| 193787|"Refutation of Al...|  (no genres listed)|       2.0|
+-------+--------------------+--------------------+----------+



In [ ]:
from pyspark.sql.functions import *

In [ ]:
newDf = dt_clean.withColumn('title', regexp_replace('title', '"', ''))

In [ ]:
newDf.where(newDf["title"].like('%"%')).show()

+-------+-----+------+----------+
|movieId|title|genres|avg_rating|
+-------+-----+------+----------+
+-------+-----+------+----------+



In [ ]:
newDf.coalesce(1).write.csv('Cleaned_Data_KS02.csv', header = True)

In [ ]:
newDf.where(newDf["title"].like("%'%")).show()

+-------+--------------------+--------------------+----------+
|movieId|               title|              genres|avg_rating|
+-------+--------------------+--------------------+----------+
|     62|Mr. Holland's Opu...|               Drama|      3.75|
|     63|Don't Be a Menace...|        Comedy|Crime|      3.12|
|     81|Things to Do in D...| Crime|Drama|Romance|      3.27|
|     82|Antonia's Line (A...|        Comedy|Drama|      4.01|
|    114|Margaret's Museum...|               Drama|      3.63|
|    117|Young Poisoner's ...|         Crime|Drama|      3.59|
|    124|Star Maker, The (...|               Drama|      3.49|
|    128|Jupiter's Wife (1...|         Documentary|      3.48|
|    209|White Man's Burde...|               Drama|      2.82|
|    258|Kid in King Arthu...|Adventure|Childre...|      2.63|
|    264|    Enfer, L' (1994)|               Drama|      3.33|
|    273|Mary Shelley's Fr...| Drama|Horror|Sci-Fi|      3.12|
|    281|Nobody's Fool (1994)|Comedy|Drama|Romance|    